### 기본 Library 선언 및 Tensorflow 버전 확인

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

print(tf.__version__)

tf.random.set_seed(777)  # for reproducibility

2.3.0-rc0


# Softmax_cross_entropy_with_logits
06-1Softmax Classifier.ipynb에서 구현한 hypothesis, cost function을 좀더 편리한 방법으로 정의해볼까 한다.<br>  
tf.nn.softmax_cross_entropy_with_logits_v2() 은 deprecate 되었다...<br>
tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, from_logits=True) 로 변경

## Softmax function

linear한 결과 값인 XW + b를 0~1사이의 값으로 변환

$$
\begin{align}
S(y_{i}) & = \frac{e^{y_{i}}}{\sum_{j} e^{y_{i}}}  \\\\\
\end{align}
$$

## Cross entropy cost / loss

\begin{align}
D(S,L)& = -{\sum_{i}L_{i}log(S_{i}) }  \\\\\
\end{align}

\begin{align}
L_{ce}& = \frac{1}{N}{\sum_{i} D(S(WX_{i}+b)L_{i}})  \\\\\
\end{align}



이전 코드에서 구현한 hypothesis 는 다음과 같다.
``` python
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logit)
```
그리고 이를 활용한 cost function은 다음과 같다.
```python
# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
```
수식을 그래로 구현하면 되지만 좀 더 간소화시켜서 구현할 수 있는 것을 Tensorflow는 제공하고 있다.

```python
# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot) # deprecate되었다...
cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, from_logits=True) # 해당 방법으로 수정

cost = tf.reduce_mean(cost_i)
```

Y_one_hot은 제공되는 Y(실제 값)가 one_hot 속성을 갖기 때문에 이렇게 이름을 붙였다.

# Softmax Zoo_classifier

* Softmax를 사용하여 Zoo 데이터를 활용하여 분류를 진행.

## Data load

![shape.jpg](자료img/animal_data_shape.jpg) 
python numpy 슬라이싱 참고 https://076923.github.io/posts/Python-numpy-5/  
xy[:,:1] # [: , :] [ 행 , 열]  
xy[:1] # [행]

In [63]:
xy = np.loadtxt("data/data-04-zoo.csv", delimiter=",", dtype=np.float32)
x_data = xy[: , 0:-1]
y_data = xy[:,[-1]]# 2차원으로 데이터를 가져옴

print(x_data.shape, y_data.shape)
print(y_data[:5])

(101, 16) (101, 1)
[[0.]
 [0.]
 [3.]
 [0.]
 [0.]]


## tf.one_hot and reshape
y_data를 살펴보면 0~6으로 7종류로 동물을 구분하고 있다. 하지만 우리가 필요한 one_hot으로 되어있는 값이다.  
그래서 tensorflow는 one_hot으로 바꿔주는 tf.one_hot()을 제공한다.  

예제를 통해 설명해보자면...
```python
nb_classes = 7
Y = tf.placeholder(tf.int32, [None, 1]) # 0 ~ 6, shape=(?, 1)
```
일단 y_data는 shape = (?, 1)임을 알 수가 있다.  
```python
Y_one_hot = tf.one_hot(Y, nb_classes) # one hot shape=(?, 1, 7)
```
tf.one_hot()의 인자로 데이터, 클래스 개수를 넘겨준다.  
하지만 이때 문제가 발생하는데 one_hot의 다음과 같은 속성 때문이다.<br><br>

**<center>If the input indices is rank N, the output will have rank N+1. The new axis is<br>    
created at dimension axis (default: the new axis is appended at the end).</center>**<br><br>
쉽게 설명하자면 차원을 1차원 더 크게 만들어 준다는 의미이다. 즉, 1차원을 넣으면 2차원을 반환

위의 animal 데이터를 예시로 들자면 y_data는 다음과 같은 형태이다.
<center>[[0.],[3.]]<br>shape = (?, 1)</center><br>
근데 이 데이터를 one_hot을 해버리면 차원을 더하면서 다음과 같은 형태로 바뀐다.  
<center>[[[1. 0. 0. 0. 0. 0. 0.]], [[0. 0. 0. 1. 0. 0. 0.]]]<br>shape = (?, 1, 7)</center><br>
하지만 이러한 형태는 우리가 원하는 형태가 아니다. 그래서 이를 해결할 방법이 tf.reshape()이다.<br>

```python
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # shape=(?, 7)
```
첫 번째 인자로 변환할 배열을 넣어준다.<br>
두 번째 인자는 변환할 형태를 정의해주면 된다. [ -1:모든 부분,  클래스 개수]<br><br>
그래서 reshpe을 거친 데이터는 다음과 같은 모양이 될것이다.
<center>[[1. 0. 0. 0. 0. 0. 0.]], [[0. 0. 0. 1. 0. 0. 0.]]<br>shape = (?, 7)</center><br>

### tf.one_hot

In [55]:
nb_classes = 7 # 0 ~ 6
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)
print(Y_one_hot.shape, "\n",Y_one_hot[:5])

(101, 7) 
 tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]], shape=(5, 7), dtype=float32)


### tf.reshape

In [48]:
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # shape=(?, 7)
print(Y_one_hot.shape, "\n", Y_one_hot[:5])

(101, 7) 
 tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]], shape=(5, 7), dtype=float32)


추후 one_hot encoding만 따로 정리하겠습니다.

## Softmax Zoo_classifier 전체 코드
위에서 y_data를 가져올 때 y_data = xy[:,[-1]] 와 같이 진행하였다. 이렇게 가져오면 2차원으로 데이터를 가져오는 꼴이고 나중에 reshape해줘야하는 불편 사항이 존재하였다. (모두의 딥러닝 시즌1에선 이런식으로 진행)<br><br>
이 부분은 수정하여 전체 코드를 진행하겠다.

In [94]:
xy = np.loadtxt("data/data-04-zoo.csv", delimiter=",", dtype=np.float32)
x_data = xy[: , 0:-1]
y_data = xy[:,-1] # 1차원으로 데이터를 가져옴

print(x_data.shape, y_data.shape)
print(y_data[:5])

(101, 16) (101,)
[0. 0. 3. 0. 0.]


In [95]:
nb_classes = 7  # 0 ~ 6

# Make Y data as onehot shape
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)
print(Y_one_hot[:5])

(101, 16) (101, 7)
tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]], shape=(5, 7), dtype=float32)


In [113]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_fn(X): # scroe : 실수 값
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

##### 왜 굳이 logit_fn(), hypothesis()를 구분해서 정의하였을까?
logit은 categorical_crossentropy()에서 활용하기 위해 정의하였다고 납득할 수 있다. 그렇다면 hypothesis는?<br>
이는 이후에 학습을 하면서 정확도를 맞추기 위한 도구로 사용이 된다.

In [ ]:
def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, from_logits=True)
    cost = tf.reduce_mean(cost_i) 
    return cost

def cost_fn2(X, Y):
    logits = hypothesis(X)
    cost_i = -tf.reduce_sum(Y* tf.math.log(logits), axis=1)
    cost = tf.reduce_mean(cost_i) 
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        loss2 =  cost_fn2(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

**위 코드의 cost_fn, cost_fn2는 같은 동작을 함.**

In [ ]:
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1)) # Y 와 예측 값을 비교
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

prediction 함수에서 위에서 정의했던 hypothesis를 사용하여 정확도를 비교한다.

## Train

In [115]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('epochs: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

epochs: 1 Loss: 0.10128264874219894, Acc: 0.9900990128517151
epochs: 100 Loss: 0.09306582808494568, Acc: 1.0
epochs: 200 Loss: 0.08603505790233612, Acc: 1.0
epochs: 300 Loss: 0.08000823110342026, Acc: 1.0
epochs: 400 Loss: 0.07478483021259308, Acc: 1.0
epochs: 500 Loss: 0.07021408528089523, Acc: 1.0
epochs: 600 Loss: 0.0661807656288147, Acc: 1.0
epochs: 700 Loss: 0.06259509176015854, Acc: 1.0
epochs: 800 Loss: 0.059386301785707474, Acc: 1.0
epochs: 900 Loss: 0.056497715413570404, Acc: 1.0
epochs: 1000 Loss: 0.053883470594882965, Acc: 1.0
